# Matrix Decompositions
## 4.1 Matrix and Trace
### Theorem 4.1. For any square matrix $A\in\mathbb{R}^{n \times n}$ A is invirtible if $\det{A}\ne0$

In [251]:
import torch
def matrix(size=(2,2),square=True):
    torch.manual_seed(17)
    if square:
        return torch.rand(size=size)
    else:
        return torch.rand(size=size)

def is_invertible(A):
    try:
        if A.shape[0] != A.shape[1]:
            print("Matrix must be square to be invertible!")
            return False
        det = torch.linalg.det(A)
        if det != 0:
            print("Is invertible")
            return True
        else:
            print("Matrix is singular (determinant is zero)")
            return False
    except Exception as e:
        print(f"Error checking invertibility: {e}")
        return False

print(is_invertible(matrix(square=True)))
print(is_invertible(matrix(square=False)))

Is invertible
True
Is invertible
True


The determinant is the product of the diagonal elements, a square matrix $T$ has a upper-triangular matrix if $T_{ij}=0$ for $i>j$ (i.e., the matrix is zero below its diagonal). The lower-trangular matrix is a matrix with zeros above its diagonal, for a triangular matrix $T\in \mathbb{R}^{n\times n}$ the determinant is the product of the diagonal elements: $$\det(T)=\prod_iT_{ii}$$
This also can be found with its eigenvalues.

In [252]:
Tu = torch.tensor([[1.0, 2.0, 3.0], # Upper Triangular
                  [0.0, 4.0, 5.0],
                  [0.0, 0.0, 6.0]])
A = matrix(square=True)
def all_close(a, b):
    return torch.allclose(a, b)
def _det_eig(A):
    return torch.prod(torch.linalg.eigvals(A)).real
# Triangular Matrix
print(all_close(torch.prod(torch.diag(Tu)), torch.linalg.det(Tu)))  
# Eigenvalues (this holds for all square matrices)
print(all_close(torch.prod(_det_eig(A)), torch.linalg.det(A)))

True
True


### Theorem 4.2. (Laplace Expansion). $A\in \mathbb{R}^{n\times n}$. $\forall j \in {1,...,n}$:
1. Expansion along column $j$
$$\det{A}=\sum_k(-1)^{k+j}a_{kj}\det(A_{k,j})$$
2. Expansion along row $j$
$$\det{A}=\sum_k(-1)^{j+k}a_{jk}\det(A_{j,k})$$
Here $A_{k,j}\in\mathbb{R}^{(n-1)\times (n-1)}$ is the submatrix of $A$ that we obtain when deleting row $k$ and column $j$. This what torch does naturally.

In [253]:
def _det_simple(A):
    if A.shape != (2,2):
        raise ValueError('Input must be a 2x2 matrix')
    return A[0, 0] * A[1, 1] - A[0, 1] * A[1, 0]

def _det_expansion(A):
    '''Expansion to square matrices with n>2'''
    n = A.shape[0]
    if n == 1:
        return A[0,0]
    elif n == 2: 
        return _det_simple(A)
    det = 0 
    for j in range(n):
        minor = torch.cat([
            A[1:, :j],
            A[1:, j+1:]
        ], dim=1)
        det += (-1)**j * A[0, j] * _det_expansion(minor)
    return det
print(all_close(_det_expansion(A), torch.linalg.det(A))) 
B = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]], dtype=torch.float32) # Singular Matrix
if _det_expansion(B) == 0:
    print(f"B is a Singular Matrix")

True
B is a Singular Matrix


- $\det(AB)=\det(A)\det(B)$
- $\det(A)=\det(A)^T$
- If A is regular $\det(A^{-1})=\frac{1}{\det(A)}$
- Determinant is invariant to the choice of basis of a linear mapping
- Adding multiple of a column/row to another one does not change $\det(A)$ 
- Multiplication of a column/row with $\lambda\in\mathbb{R}$ scales $\det(A)$ by $\lambda$. In particular, $\det(\lambda A)=\lambda^n\det(A)$
- Swapping two rows/columns changes the sign of $\det(A)$

### Definition 4.4. The trace of a matrix $A\in\mathbb{R}^{n\times n}$ 
Is defined as $$\text{tr}(A):=\sum_i^na_{ii}$$
The trace is invariant under cyclic permutations $$\text{tr}(AKL)=\text{tr}(KLA)$$

In [254]:
def _trace(A):
    a = torch.diag(A)
    return torch.sum(a)
print(_trace(A) == torch.trace(A)) # Definition of Trace operation
a = 3; k = 2; l = 4
A = matrix(size=(a, k))
K = matrix(size=(k, l))
L = matrix(size=(l, a))
def cyclic_perm(A, K, L):
    return torch.trace(A @ K @ L)
print(all_close(cyclic_perm(A,K,L), cyclic_perm(K,L,A)))

tensor(True)
True


### Definition 4.5 (Characteristic Polynomial). For $\lambda\in \mathbb{R}$ and a square matrix
We have 
$$
\begin{align}
p_A(\lambda) &:= \det(A - \lambda I) \\
             &= c_0 + c_1\lambda + c_2\lambda^2 + \cdots + c_{n-1}\lambda^{n-1} + (-1)^n\lambda^n
\end{align}
$$
This is the characteristic polynomial of $A$.

In [255]:
A = matrix(square=True)
lmd = torch.tensor([[2]])
def char_poly(A, lmd):
    n, m = A.shape[0], A.shape[1]
    I = torch.eye(n, m)
    return torch.det(A - lmd*I)
char_poly(A, lmd)

tensor(2.4933)

This allow us to compute eigenvalues and eigenvectors 

## Eigenvalues and Eigenvectors
Every linear mapping has a unique transformation matrix given an ordered basis.
We can interpret linear mapping and their associated transformation by performing an "eigen" analysis.
### Definition 4.6. $A\in\mathbb{R}^{n\times n}$. Then $\lambda\in\mathbb{R}$ is an eigenvalue of A and $x\in\mathbb{R}^n\{0\}$
The eigenvector then is $Ax=\lambda x$, often the eigenvectors are sorted in descending order.

### Definition 4.14 $A\in\mathbb{R}^{m\times n}$, we can always obtain a symmetric, positive semidefinite matrix $S\in\mathbb{R}^{n\times n}$
We define $$S:=A^TA$$
If $rk(A)=n$, then $S:=A^TA$ is symmetric, positive definite.


In [256]:
def _symmetry(A):
    return A.T @ A # Dot Product
A = matrix(size=(2,2))
S = _symmetry(A)
print(all_close(S, S.T))
print(all_close(S, A.T @ (A.T).T))
print(all_close(S, S.T))

True
True
True


### Theorem 4.15 (Spectral Theorem). $A\in\mathbb{R}^{n\times n}$ is symmetric.
There exists an orthonormal basis of the corresponding vector space V consisting of eigenvectors of A, and each eigenvalue is real.

In [257]:
def _spectral(A):
    assert A.shape[0] == A.shape[1], "A must be Square" # square
    D, P = torch.linalg.eigh(A) # D:eigenvalues, P:eigenvectors
    if not all_close(A, A.T):
        return False
    return all_close(P @ torch.diag(D) @ P.T, A)
#  torch.cat((A, A),dim=1) for non-square
print(_spectral(S))

True


### Theorem 4.17. The trace of a matrix $A\in\mathbb{R}^{n\times n}$ is the sum of the eigenvalues
$$tr(A)=\sum_i\lambda_i$$
Where $\lambda_i\in \mathbb{C}$ are (possibly repeated) eigenvalues of $A$.

In [258]:
def _trace(A):
    eigs = torch.linalg.eigvalsh(A)
    return torch.sum(eigs)
all_close(_trace(A), torch.trace(A)) 

True

## Cholesky Decomposition
### Theorem 4.18 (Cholesky Decomposition). For any symmetric $A\in R^{n\times n},\forall x\in \mathbb{R}\ne0$ and $x^TAx>0$

In [259]:
def _triang(A):
    '''Lower triangular matrix'''
    Ll = torch.einsum('ij,ij->ij', A, torch.tril(torch.ones_like(A)))
    return Ll
def _cholesky(A):
    n = A.size(0)
    L = torch.zeros_like(A)
    for j in range(n):
        for k in range(j+1):
            s = torch.sum(L[j, :k]*L[k, :k])
            if j == k:
                # Diagonal element
                L[j,j] = torch.sqrt(A[j,j]-s)
            else:
                # Off-diagonal element
                L[j,k] = (A[j,k] - s) / L[k,k]
    return L
A = matrix(size=(3,3))
S = _symmetry(A) # Symmetric Positive Definite Matrix
print(_cholesky(S))
print(all_close(_cholesky(S), torch.cholesky(S)))
%timeit _cholesky(S)
%timeit torch.cholesky(S)

tensor([[ 0.5918,  0.0000,  0.0000],
        [ 0.7519,  0.1425,  0.0000],
        [ 1.0269, -0.3922,  0.0464]])
True
430 μs ± 14.5 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
30.2 μs ± 3 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## 4.4 Eigendecomposition and Diagonalization
The eigenvalue decomposition requires square matrices. It would be
useful to perform a decomposition on general matrices.

## 4.5 Singular Value Decomposition
Fundamental theorem of linear algebrea, because it can be applied to all matrices, not only to square matrices.
### Theorem 4.22. (SVD Theorem). $A\in\mathbb{R}^{m\times n}$ with $r\in[0,\min(m,n)]$.
The SVD of A is a decomposition.

In [260]:
A = matrix(size=(3, 5))
print(A.size())
U, S, V = torch.svd(A)

torch.Size([3, 5])


## 4.6 Matrix Approximation
### Definition 4.23 (Spectral Norm of a Matrix). For $x\in\mathbb{R}^n\{0\}$
The spectral norm is $$||A||_2:=\max_x\frac{||Ax||_2}{||x||_2}$$
Which can also be written as $$||A||_2=\sigma_{max}(A)$$
the largest singular value of the SVD.

In [261]:
def _spectral_norm(A):
    _, S, _ = torch.svd(A)
    return S[0]
A = matrix(size=(3, 5))
_spectral_norm(A)

tensor(1.4816)

## Exercises
1. Compute the determinant using the Laplace expansion (using the first row) and the Sarrus rule 

In [ ]:
def _det_expansion(A):
    '''Expansion to square matrices with n>2'''
    n = A.shape[0]
    if n == 1:
        return A[0,0]
    elif n == 2: 
        return _det_simple(A)
    det = 0 
    for j in range(n):
        minor = torch.cat([
            A[1:, :j],
            A[1:, j+1:]
        ], dim=1)
        det += (-1)**j * A[0, j] * _det_expansion(minor)
    return det

A = torch.tensor([[[1.,3.,5.],[2.,4.,6.],[0.,2.,4.]]])
%timeit _det_expansion(A)
%timeit torch.linalg.det(A)

7.17 μs ± 170 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
56 μs ± 1.45 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
